In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install fire
!pip install lmdb
!pip install opencv-python
!pip install natsort
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=9866b9211d6f920f4531c1af17eb38d6c00a46a51a6298af2890ab497d7b3caa
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 2.3 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495
Receiving objects: 100% (499/499), 3.07 MiB | 15.20 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


### Paste the code to deep-text-recognition-benchmark/create_lmdb_dataset.py

https://freedium.cfd/https://pub.towardsai.net/how-to-fine-tune-easyocr-to-achieve-better-ocr-performance-1540f5076428#:~:text=import%20fire%0Aimport,fire.Fire(createDataset)

In [5]:
import cv2
import numpy as np
import pandas as pd
import random
import os

In [18]:
def generate_numberplate_image(text, save_path):
    height, width = 100, 300
    white_bg = np.ones((height, width, 3), dtype=np.uint8) * 255

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    font_color = (0, 0, 0)

    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    x = random.randint(0, width - text_size[0])
    y = random.randint(text_size[1], height)
    org = (x, y)

    image_with_text = cv2.putText(white_bg.copy(), text, org, font, font_scale, font_color, font_thickness, cv2.LINE_AA)

    angle = random.uniform(-15, 15)
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    image_with_text = cv2.warpAffine(image_with_text, rotation_matrix, (width, height), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

    kernel_size = random.choice([3, 5, 7])
    image_with_text = cv2.GaussianBlur(image_with_text, (kernel_size, kernel_size), 0)

    cv2.imwrite(save_path, image_with_text)

In [34]:
output_dir = r'generated_images'
os.makedirs(output_dir, exist_ok=True)
labels_file_path = os.path.join(output_dir, 'labels.txt')

In [35]:
height, width = 30, 90

In [36]:
image_data = []

In [37]:
with open(labels_file_path, 'w') as labels_file:
  for i in range(12000):
      state_code = random.choice(['MH', 'GJ', 'DL', 'KA', 'TN'])
      number_part1 = str(random.randint(10, 99))
      alphabets = ''.join(random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') for _ in range(2))
      number_part2 = str(random.randint(1000, 9999))

      numberplate_text = f'{state_code} {number_part1} {alphabets} {number_part2}'

      image_path = os.path.join(output_dir, f'image_{i + 1}.png')

      generate_numberplate_image(numberplate_text, image_path)

      image_data.append({'image_path': f'image_{i + 1}.png', 'number_plate_text': numberplate_text})
      labels_file.write(f'image_{i + 1}.png {numberplate_text}\n')

print("Images generated successfully.")
df = pd.DataFrame(image_data)
csv_file_path = os.path.join("/content/", 'image_data.csv')
df.to_csv(csv_file_path, index=False)

Images generated successfully.


In [24]:
# !rm -rf generated_images/
# !rm -rf lmdb_dataset/

In [38]:
!python /content/deep-text-recognition-benchmark/create_lmdb_dataset.py /content/generated_images/ /content/generated_images/labels.txt /content/lmdb_dataset

1200
Written 1000 / 1200
Created dataset with 1200 samples


In [29]:
# !rm -rf saved_models/

In [30]:
!python /content/deep-text-recognition-benchmark/train.py --train_data /content/lmdb_dataset --valid_data /content/lmdb_dataset --select_data "/" --batch_ratio 1.0 --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --batch_size 32 --data_filtering_off --workers 4 --batch_max_length 10 --num_iter 100 --valInterval 5 --saved_model "/content/drive/MyDrive/BE Project/Data/EasyOCR/Base Model/TPS-ResNet-BiLSTM-Attn.pth"

--------------------------------------------------------------------------------
dataset_root: /content/lmdb_dataset
opt.select_data: ['/']
opt.batch_ratio: ['1.0']
--------------------------------------------------------------------------------
dataset_root:    /content/lmdb_dataset	 dataset: /
sub-directory:	/.	 num samples: 12000
num total samples of /: 12000 x 1.0 (total_data_usage_ratio) = 12000
num samples of / per batch: 32 x 1.0 (batch_ratio) = 32
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
-----------------------------------------------------------------

In [41]:
!python /content/deep-text-recognition-benchmark/demo.py --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --image_folder /content/test_images --saved_model /content/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from /content/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
/content/test_images/image_1.png	gj10zs6717               	0.7986
/content/test_images/image_2.png	ka64lu3222           

In [33]:
import os
import shutil

source_dir = '/content/saved_models'
dest_dir = '/content/drive/MyDrive/BE Project/Data/EasyOCR/Trained Model 2024-02-23-8:40'

if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
        shutil.copytree(source_dir, dest_dir)

In [40]:
!python /content/deep-text-recognition-benchmark/test.py --eval_data /content/test_lmdb_dataset --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --saved_model /content/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth --batch_max_length 10 --workers 4 --batch_size 32 --data_filtering_off

model input parameters 32 100 20 1 512 256 38 10 TPS ResNet BiLSTM Attn
loading pretrained model from /content/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth
dataset_root:    /content/test_lmdb_dataset	 dataset: /
sub-directory:	/.	 num samples: 1200
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
85.333
